# M45

Small open economy model with a VER quota


|Markets   |   X1  |    X2  |    E1  |    M2  |    W  | CONSH | CONSF   |
|----------|-------|--------|--------|--------|-------|-------|---------|
|P1        |  150  |        |   -50  |        |   -90 |       |   -10   |
|P2        |       |    40  |        |    60  |  -100 |       |         | 
|PL        | -100  |   -20  |        |        |       |   120 |         |  
|PK        |  -50  |   -20  |        |        |       |    70 |         |  
|PW        |       |        |        |        |   190 |  -190 |         | 
|PFX       |       |        |    50  |   -50  |       |       |         |  
|Q (ver)   |       |        |        |   -10  |       |       |    10   |

In [1]:
using MPSGE_MP

In [5]:
M45 = MPSGEModel()


initial_tax = .2

@parameters(M45, begin
    PE1, 1,     (description = "Export price of good 1",)
    PE2, 1/(1+initial_tax)*.99,   (description = "Export price of good 2",)
    PM1, 1.01,  (description = "Import price of good 1",)
    PM2, 1/(1+initial_tax),     (description = "Import price of good 1",)    
    TM2, initial_tax, (description = "Tariff for good 2",)
end)

@sectors(M45, begin
    X1,	(description = "Production index for good 1",)
    X2,	(description = "Production index good 2",)
    E1,	(description = "Export level of good 1",)
    E2,	(description = "Export level of good 2",)
    M1,	(description = "Import level of good 1",)
    M2,	(description = "Import level of good 2",)
    W,	(description = "Welfare index ",)
end)
@commodities(M45, begin
    P1,	(description = "Price index for good 1",)
    P2,	(description = "Price index for good 1",)
    PFX,	(description = "Read exchange rate index",)
    PW,	(description = "Welfare price index",)
    PL,	(description = "Wage index",)
    PK,	(description = "Capital rental index",)
end)
@consumers(M45, begin
    CONSH,	(description = "Income level for domestic consumer",)
    CONSF,  (description = "Income level for foreign consumer (quota holder)",)
end)

@auxiliaries(M45, begin
    V,  (description = "Endogenous tax, shadow tax for VER",)
    Q,  (description = "Endogenous tax, shadow tax for quota",)

end)


@production(M45, X1, [s=1,t=0],begin
    @output(P1, 150, t)
    @input(PL, 100,s)
    @input(PK, 50, s)
end)

@production(M45, X2, [s=1,t=0], begin
    @output(P2, 40, t)
    @input(PL, 20, s)
    @input(PK, 20, s)
end)

@production(M45, E1, [s=0,t=0], begin
    @output(PFX, 50*PE1, t)
    @input(P1, 50, s)
end)

@production(M45, M2, [s=0,t=0], begin
    @output(P2, 60, t)
    @input(PFX, 60*PM2, s, taxes = [Tax(CONSF, V), Tax(CONSH, Q)]) # what is N?
end)

@production(M45, E2, [s=0,t=0], begin
    @output(PFX, 60*PE2, t)
    @input(P2, 60, s)
end)

@production(M45, M1, [s=0,t=0], begin
    @output(P1, 50, t)
    @input(PFX, 50*PM1, s)
end)

@production(M45, W, [s=1,t=0], begin
    @output(PW, 190, t)
    @input(P1,  90, s)
    @input(P2, 100, s)
end)

@demand(M45, CONSH, begin
    @final_demand(PW, 190)
end,begin
    @endowment(PL, 120)
    @endowment(PK, 70)
end);

@demand(M45, CONSF, begin
    @final_demand(P1, 10)
end, begin

end)


@aux_constraint(M45, Q,
    1 - M2
)

@aux_constraint(M45, V,
    1 - M2
);

# Benchmark

In [6]:
fix(PW, 1)

set_start_value(E2, 0)
set_start_value(M1, 0)
set_start_value(V, 0.2)
set_start_value(Q, 0)

solve!(M45, cumulative_iteration_limit = 0)
generate_report(M45)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_79DF.tmp
 > cumulative_iteration_limit 0
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris
Preprocessed size   : 16

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     1     1 0.0000e+00           I 0.0e+00 0.0e+00 (zero_profit[X1)

Major Iterations. . . . 0
Minor Iterations. . . . 0
Restarts. . . . . . . . 0
Crash Iterations. . . . 0
Gradient Steps. . . . . 0
Function Evaluations. . 1
Gradient Evaluations. . 1
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.079000
Residual. . . . . . . . 0.000000e+00
Postsolved residual: 0.0000e+00


Solver Status: LOCALLY_SOLVED
Model Status: FEASIBLE_POINT

Row,var,value,margin
,GenericV…,Float64,Float64
1,X2,1.0,0.0
2,W,1.0,0.0
3,M2,1.0,0.0
4,X1,1.0,0.0
5,PL,1.0,0.0
6,PW,1.0,0.0
7,P1,1.0,0.0
8,PK,1.0,0.0
9,P2,1.0,0.0


# Counterfactual

In [8]:
fix(V, 0)

solve!(M45)
generate_report(M45)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_FEC0.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 15

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             1.9930e+01             0.0e+00 (zero_profit[M2)
    1    10     1    12 1.9892e+01  1.9e-03    0.0e+00 (zero_profit[M2)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0    11     2 1.9892e+01           I 0.0e+00 1.9e+01 (zero_profit[M2)
    1     1    12     3 1.2968e-01  1.0e+00 SM 0.0e+00 1.3e-01 (income_balance[)
    2     1    13     4 5.2430e-07  1.0e+00 SO 0.0e+00 4.1e-07 (market_clearanc)

Major Iterations. . . . 2
Minor Iterations. . . . 2
Restarts. . . . . . . . 0
Crash Iterations. . . . 1
Gradient Steps. . . . . 0
Fun

Row,var,value,margin
,GenericV…,Float64,Float64
1,X2,1.1274,-2.35366e-8
2,W,1.05259,-6.02839e-10
3,M2,1.0,-2.0205e-8
4,X1,0.965971,-7.82003e-8
5,PL,0.991785,-3.15005e-7
6,PW,1.0,-2.05442e-7
7,P1,0.998283,1.0572e-8
8,PK,1.01141,4.09841e-7
9,P2,1.00155,-9.90397e-9
